# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)



## Generate Cities List

In [9]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []



# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

616

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [19]:
url = "http://api.openweathermap.org/data/2.5/weather?"
query_url = url + "appid=" + weather_api_key + "&q=" + city

city_weather = []
cloudiness = []
date = []
humidity = []
lat = []
lng = []
max_temp = []
wind_speed = []
country = []


record = 0
record_set = 1


print(f'Beginning Data Retrieval')
print(f'------------------------')

for city in cities:
    record += 1
    response = requests.get(query_url)
    weather_json = response.json()
    city_weather.append(city)
    
    try:
        response = requests.get(query_url + city.replace(" ","&")).json()
        cloudiness.append(response['clouds'])
        country.append(response['sys']['country'])
        date.append(response['dt'])
        humidity.append(response['main']['humidity'])
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        max_temp.append(response['main']['temp_max'])
        wind_speed.append(response['wind']['speed'])
 
        print(f"Processing Record {record} of Set {record_set} | {city}")
    except Exception (KeyError, IndexError):
        print(f'City not found. Skipping...')
        
print(f'------------------------') 
print(f'Data Retrieval Complete')
print(f'------------------------')



Beginning Data Retrieval
------------------------


TypeError: catching classes that do not inherit from BaseException is not allowed

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:

weather_dict = {
    "City": cities,
    "Lat" : lat,
    "Lng" : lng,
    "Max Temp": max_temp,
    "Humidity": humidity,
    "Cloudiness": cloudiness,
    "Wind Speed": wind_speed,
    "Country" : country,
    "Date" : date

}
weather_data = pd.DataFrame(weather_dict)
weather_data.to_csv(index = False)

weather_data.head()



## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.
humid_city = weather_data["Humidity"] > 100
humid_city


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
clean_city_data = pd.DataFrame(weather_data["Humidity"]<100)


In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv
clean_city_data.to_csv(index = False)

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
plt.scatter(weather_data["Lat"], weather_data["Max Temp"], marker="o", edgecolors='black')

plt.title(f"City Latitude vs. Temperature")
plt.ylabel("Temperature (Fahrenheit)")
plt.xlabel("Latitude")
plt.grid(True)

plt.savefig("Figures/latitude_vs_temperature.png")

plt.show()

## Latitude vs. Humidity Plot

In [ ]:

plt.scatter(weather_data["Lat"], weather_data["Humidity"], marker="o", edgecolors='black')

plt.title(f"City Latitude vs. Humidity")
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.grid(True)

plt.savefig("Figures/latitude_vs_humidity.png")

plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:

plt.scatter(weather_data["Lat"], weather_data["Cloudiness"], marker="o", edgecolors='black')

plt.title(f"City Latitude vs. Cloudiness")
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)

plt.savefig("Figures/latitude_vs_cloudiness.png")

plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:

plt.scatter(weather_data["Lat"], weather_data["Wind Speed"], marker="o", edgecolors='black')

plt.title(f"City Latitude vs. Wind Speed")
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude")
plt.grid(True)

plt.savefig("Figures/latitude_vs_windspeed.png")

plt.show()

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots



In [ ]:
# Create Northern and Southern Hemisphere DataFrames
north_df = pd.DataFrame(weather_data["Lat"]>0)

south_df = pd.DataFrame(weather_data["Lat"]<0)

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_axis = north_df["Lat"]
y_axis = north_df["Max Temp"]

correlation = st.pearsonr(x_axis, y_axis)
(s,i,r,p,std) = linregress(x_axis, y_axis)
regress = x_axis*s + i
line_eq = "y=" + str(round(s,2))+"x+" +str(round(i,2))

plt.scatter(x_axis, y_axis, marker="o", edgecolors='black')
plt.plot(x_axis, regress, "r-")
plt.annotate(line_eq,(10,0), fontsize=15, color = "red")
plt.title(f"Nothern Hemisphere Latitude vs. Temperature")
plt.ylabel("Temperature (Fahrenheit)")
plt.xlabel("Latitude")
plt.grid(True)

print(f"The r-squared is: {r**2}")
plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_axis = south_df["Lat"]
y_axis = south_df["Max Temp"]

correlation = st.pearsonr(x_axis, y_axis)
(s,i,r,p,std) = linregress(x_axis, y_axis)
regress = x_axis*s + i
line_eq = "y=" + str(round(s,2))+"x+" +str(round(i,2))

plt.scatter(x_axis, y_axis, marker="o", edgecolors='black')
plt.plot(x_axis, regress, "r-")
plt.annotate(line_eq,(10,0), fontsize=15, color = "red")
plt.title(f"Southern Hemisphere Latitude vs. Temperature")
plt.ylabel("Temperature (Fahrenheit)")
plt.xlabel("Latitude")
plt.grid(True)

print(f"The r-squared is: {r**2}")
plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_axis = north_df["Lat"]
y_axis = north_df["Humidity"]

correlation = st.pearsonr(x_axis, y_axis)
(s,i,r,p,std) = linregress(x_axis, y_axis)
regress = x_axis*s + i
line_eq = "y=" + str(round(s,2))+"x+" +str(round(i,2))

plt.scatter(x_axis, y_axis, marker="o", edgecolors='black')
plt.plot(x_axis, regress, "r-")
plt.annotate(line_eq,(10,0), fontsize=15, color = "red")
plt.title(f"Nothern Hemisphere Latitude vs. Humidity")
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.grid(True)

print(f"The r-squared is: {r**2}")
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_axis = south_df["Lat"]
y_axis = south_df["Humidity"]

correlation = st.pearsonr(x_axis, y_axis)
(s,i,r,p,std) = linregress(x_axis, y_axis)
regress = x_axis*s + i
line_eq = "y=" + str(round(s,2))+"x+" +str(round(i,2))

plt.scatter(x_axis, y_axis, marker="o", edgecolors='black')
plt.plot(x_axis, regress, "r-")
plt.annotate(line_eq,(10,0), fontsize=15, color = "red")
plt.title(f"Southern Hemisphere Latitude vs. Humidity")
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.grid(True)

print(f"The r-squared is: {r**2}")
plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_axis = north_df["Lat"]
y_axis = north_df["Cloudiness"]

correlation = st.pearsonr(x_axis, y_axis)
(s,i,r,p,std) = linregress(x_axis, y_axis)
regress = x_axis*s + i
line_eq = "y=" + str(round(s,2))+"x+" +str(round(i,2))

plt.scatter(x_axis, y_axis, marker="o", edgecolors='black')
plt.plot(x_axis, regress, "r-")
plt.annotate(line_eq,(10,0), fontsize=15, color = "red")
plt.title(f"Nothern Hemisphere Latitude vs. Cloudiness")
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)

print(f"The r-squared is: {r**2}")
plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_axis = south_df["Lat"]
y_axis = south_df["Cloudiness"]

correlation = st.pearsonr(x_axis, y_axis)
(s,i,r,p,std) = linregress(x_axis, y_axis)
regress = x_axis*s + i
line_eq = "y=" + str(round(s,2))+"x+" +str(round(i,2))

plt.scatter(x_axis, y_axis, marker="o", edgecolors='black')
plt.plot(x_axis, regress, "r-")
plt.annotate(line_eq,(10,0), fontsize=15, color = "red")
plt.title(f"Southern Hemisphere Latitude vs. Cloudiness")
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)

print(f"The r-squared is: {r**2}")
plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_axis = north_df["Lat"]
y_axis = north_df["Wind Speed"]

correlation = st.pearsonr(x_axis, y_axis)
(s,i,r,p,std) = linregress(x_axis, y_axis)
regress = x_axis*s + i
line_eq = "y=" + str(round(s,2))+"x+" +str(round(i,2))

plt.scatter(x_axis, y_axis, marker="o", edgecolors='black')
plt.plot(x_axis, regress, "r-")
plt.annotate(line_eq,(10,0), fontsize=15, color = "red")
plt.title(f"Nothern Hemisphere Latitude vs. Wind Speed")
plt.ylabel("Wind Speed (%)")
plt.xlabel("Latitude")
plt.grid(True)

print(f"The r-squared is: {r**2}")
plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_axis = south_df["Lat"]
y_axis = south_df["Wind Speed"]

correlation = st.pearsonr(x_axis, y_axis)
(s,i,r,p,std) = linregress(x_axis, y_axis)
regress = x_axis*s + i
line_eq = "y=" + str(round(s,2))+"x+" +str(round(i,2))

plt.scatter(x_axis, y_axis, marker="o", edgecolors='black')
plt.plot(x_axis, regress, "r-")
plt.annotate(line_eq,(10,0), fontsize=15, color = "red")
plt.title(f"Southern Hemisphere Latitude vs. Wind Speed")
plt.ylabel("Wind Speed (%)")
plt.xlabel("Latitude")
plt.grid(True)

print(f"The r-squared is: {r**2}")
plt.show()